# Module 12.4: Tenant Lifecycle Management (L3)

**Level 3 - SaaS Operations & Monetization**

This notebook demonstrates complete tenant lifecycle management including:
- Plan upgrades and downgrades with billing integration
- GDPR-compliant data exports
- Soft-deletion with retention policies
- Reactivation workflows

**Duration**: 35 minutes  
**Difficulty**: Advanced

## Setup and Imports

In [ ]:
# Import required modules
import json
import sys
from pathlib import Path
from datetime import datetime, timedelta

# Add parent directory to path for imports
sys.path.insert(0, str(Path(__file__).parent.parent) if '__file__' in globals() else '..')

# Import our module
import src.l3_m12_tenant_lifecycle_management as lifecycle
from config import Config, get_stripe_client

# Load example data
with open('../example_data.json') as f:
    example_data = json.load(f)

print("✓ Imports successful")
print(f"✓ Loaded {len(example_data['tenants'])} example tenants")

# Expected:
# ✓ Imports successful
# ✓ Loaded 4 example tenants

## 1. State Machine Overview

The tenant lifecycle uses an 8-state machine with validated transitions to prevent race conditions and invalid state changes.

In [ ]:
# Create a tenant and state machine
tenant = lifecycle.TenantMetadata(
    tenant_id="demo_001",
    name="Demo Corp",
    email="admin@demo.com",
    current_plan="free",
    state=lifecycle.TenantState.ACTIVE
)

sm = lifecycle.TenantLifecycleStateMachine(tenant)

# Show valid transitions from ACTIVE state
print(f"Current state: {sm.tenant.state}")
print(f"Valid transitions: {sm.VALID_TRANSITIONS[lifecycle.TenantState.ACTIVE]}")

# Test valid transition
print(f"\nCan transition to UPGRADING? {sm.can_transition(lifecycle.TenantState.UPGRADING)}")
print(f"Can transition to DELETED? {sm.can_transition(lifecycle.TenantState.DELETED)}")

# Expected:
# Current state: TenantState.ACTIVE
# Valid transitions: [TenantState.UPGRADING, TenantState.DOWNGRADING, ...]
# Can transition to UPGRADING? True
# Can transition to DELETED? False

## 2. Plan Upgrades

Upgrades provision resources **before** billing changes to avoid service interruption. Includes automatic rollback on failure.

In [ ]:
# Get a tenant on free plan
tenant_to_upgrade = example_data['tenants'][1]  # TechStart Inc on free plan
print(f"Tenant: {tenant_to_upgrade['name']}")
print(f"Current plan: {tenant_to_upgrade['current_plan']}")
print(f"Current usage: {tenant_to_upgrade['current_usage']}")

# Execute upgrade (without Stripe for demo)
result = lifecycle.upgrade_tenant(
    tenant_data=tenant_to_upgrade,
    to_plan="starter",
    plan_hierarchy=Config.PLAN_HIERARCHY,
    plan_limits=Config.PLAN_LIMITS,
    stripe_client=None  # Skip Stripe for demo
)

print(f"\n✓ Upgrade result: {result['success']}")
print(f"  New plan: {result.get('new_plan', 'N/A')}")

# Expected:
# Tenant: TechStart Inc
# Current plan: free
# ✓ Upgrade result: True
# New plan: starter

## 3. Plan Downgrades

Downgrades validate current usage fits new plan limits to prevent data loss. Can be scheduled for billing period end.

In [ ]:
# Successful downgrade (usage fits limits)
tenant_good = example_data['tenants'][0]  # Acme Corp on starter
print("=== Valid Downgrade ===")
print(f"Tenant: {tenant_good['name']}, Plan: {tenant_good['current_plan']}")
print(f"Usage: {tenant_good['current_usage']}")
print(f"Free plan limits: {Config.PLAN_LIMITS['free']}")

result_good = lifecycle.downgrade_tenant(
    tenant_data=tenant_good,
    to_plan="free",
    plan_hierarchy=Config.PLAN_HIERARCHY,
    plan_limits=Config.PLAN_LIMITS
)
print(f"Result: {result_good['success']}\n")

# Failed downgrade (usage exceeds limits)
tenant_bad = example_data['tenants'][2]  # Enterprise Solutions on professional
print("=== Blocked Downgrade ===")
print(f"Tenant: {tenant_bad['name']}, Plan: {tenant_bad['current_plan']}")
print(f"Usage: {tenant_bad['current_usage']}")
print(f"Starter plan limits: {Config.PLAN_LIMITS['starter']}")

result_bad = lifecycle.downgrade_tenant(
    tenant_data=tenant_bad,
    to_plan="starter",
    plan_hierarchy=Config.PLAN_HIERARCHY,
    plan_limits=Config.PLAN_LIMITS
)
print(f"Result: {result_bad['success']}")
print(f"Error: {result_bad.get('error', 'N/A')[:80]}...")

# Expected:
# Valid Downgrade: success=True
# Blocked Downgrade: success=False, error mentions users exceed limit

## 4. GDPR-Compliant Data Export

Chunked exports prevent memory issues. Background jobs generate signed URLs with checksums for verification.

In [ ]:
# Initiate data export
tenant_for_export = example_data['tenants'][0]
print(f"Exporting data for: {tenant_for_export['name']}")

export_job = lifecycle.export_tenant_data(
    tenant_data=tenant_for_export,
    export_type="full"
)

print(f"\n✓ Export initiated")
print(f"  Export ID: {export_job['export_id']}")
print(f"  Status: {export_job['status']}")
print(f"  Estimated completion: {export_job['estimated_completion']}")

# Simulate processing the export
service = lifecycle.DataExportService()
tenant_obj = lifecycle.TenantMetadata(**tenant_for_export)
export_result = service.process_export(tenant_obj, export_job['export_id'])

print(f"\n✓ Export completed")
print(f"  File size: {export_result['file_size_mb']} MB")
print(f"  Checksum: {export_result['checksum'][:32]}...")
print(f"  Download URL expires: {export_result['url_expires_at']}")

# Expected:
# ✓ Export initiated
# Status: queued
# ✓ Export completed
# File size: 0.0X MB

## 5. Soft Deletion with Retention

Tenants can be recovered within retention period (30-90 days). After retention expires, data is permanently deleted.

In [ ]:
# Soft delete a tenant
tenant_to_delete = {
    "tenant_id": "tenant_demo_delete",
    "name": "DeleteMe Corp",
    "email": "admin@deleteme.com",
    "current_plan": "starter",
    "state": "active"
}

print(f"Deleting tenant: {tenant_to_delete['name']}")

deletion_result = lifecycle.delete_tenant(
    tenant_data=tenant_to_delete,
    requested_by="admin@deleteme.com"
)

print(f"\n✓ Soft deletion completed")
print(f"  Deletion ID: {deletion_result['deletion_id']}")
print(f"  Status: {deletion_result['status']}")
print(f"  Retention days: {deletion_result['retention_days']}")
print(f"  Can reactivate: {deletion_result['can_reactivate']}")
print(f"  Hard delete scheduled: {deletion_result['hard_delete_scheduled_at']}")

# Simulate hard delete after retention
manager = lifecycle.DeletionManager()
tenant_obj = lifecycle.TenantMetadata(**tenant_to_delete)
tenant_obj.state = lifecycle.TenantState.DELETED

print(f"\n--- After retention period ---")
hard_delete_result = manager.hard_delete(tenant_obj)
print(f"✓ Hard delete verified: {hard_delete_result['all_verified']}")
print(f"  Verification steps: {len(hard_delete_result['verification'])}")

# Expected:
# ✓ Soft deletion completed
# Retention days: 30
# Can reactivate: True
# ✓ Hard delete verified: True

## 6. Reactivation Workflow

Win-back campaigns and recovery from suspension/deletion. Handles state conflicts and restores service.

In [ ]:
# Reactivate suspended tenant
suspended_tenant = example_data['tenants'][3]  # Suspended Co
print(f"Reactivating: {suspended_tenant['name']}")
print(f"Current state: {suspended_tenant['state']}")

workflow = lifecycle.ReactivationWorkflow()
tenant_obj = lifecycle.TenantMetadata(**suspended_tenant)

# Check if can reactivate
can_reactivate, reason = workflow.can_reactivate(tenant_obj)
print(f"Can reactivate: {can_reactivate}")

if can_reactivate:
    reactivation_result = lifecycle.reactivate_tenant(
        tenant_data=suspended_tenant,
        reactivation_plan="starter",
        stripe_client=None
    )
    
    print(f"\n✓ Reactivation successful")
    print(f"  Tenant ID: {reactivation_result['tenant_id']}")
    print(f"  Plan: {reactivation_result['plan']}")
    print(f"  Reactivated at: {reactivation_result['reactivated_at']}")

# Test reactivation within retention period
print("\n--- Reactivate soft-deleted tenant (within retention) ---")
deleted_tenant = {
    "tenant_id": "tenant_winback",
    "name": "WinBack Corp",
    "email": "admin@winback.com",
    "current_plan": "professional",
    "state": "deleted",
    "deleted_at": datetime.utcnow().isoformat()  # Just deleted
}

winback_result = lifecycle.reactivate_tenant(
    tenant_data=deleted_tenant,
    reactivation_plan="professional"
)
print(f"✓ Win-back successful: {winback_result['success']}")

# Expected:
# Can reactivate: True
# ✓ Reactivation successful
# ✓ Win-back successful: True

## 7. Common Failure Scenarios

Understanding and handling critical failure modes from the module.

In [ ]:
print("=== Failure 1: Downgrade Blocked (Usage Exceeds Limits) ===")
# Enterprise tenant trying to downgrade to starter
enterprise_tenant = example_data['tenants'][2]
result = lifecycle.downgrade_tenant(
    tenant_data=enterprise_tenant,
    to_plan="starter",
    plan_hierarchy=Config.PLAN_HIERARCHY,
    plan_limits=Config.PLAN_LIMITS
)
print(f"Success: {result['success']}")
print(f"Error: {result.get('error', 'N/A')}")

print("\n=== Failure 2: Invalid State Transition ===")
# Try to directly transition deleted tenant to active (must use reactivation)
deleted_tenant = {
    "tenant_id": "invalid_transition",
    "name": "Invalid Corp",
    "email": "admin@invalid.com",
    "current_plan": "free",
    "state": "deleted"
}
tenant_obj = lifecycle.TenantMetadata(**deleted_tenant)
sm = lifecycle.TenantLifecycleStateMachine(tenant_obj)
success = sm.transition(lifecycle.TenantState.ACTIVE)
print(f"Can transition DELETED -> ACTIVE: {success}")
print(f"Valid transitions from DELETED: {sm.VALID_TRANSITIONS[lifecycle.TenantState.DELETED]}")

print("\n=== Failure 3: Reactivation After Retention Period ===")
# Tenant deleted 91 days ago (past 30-day retention)
expired_tenant = {
    "tenant_id": "expired",
    "name": "Expired Corp",
    "email": "admin@expired.com",
    "current_plan": "starter",
    "state": "deleted",
    "deleted_at": (datetime.utcnow() - timedelta(days=91)).isoformat()
}
workflow = lifecycle.ReactivationWorkflow()
tenant_obj = lifecycle.TenantMetadata(**expired_tenant)
can_reactivate, reason = workflow.can_reactivate(tenant_obj)
print(f"Can reactivate: {can_reactivate}")
print(f"Reason: {reason}")

print("\n✓ All failure scenarios demonstrated")

# Expected:
# Failure 1: Error mentions users exceed limit
# Failure 2: Can transition: False
# Failure 3: Can reactivate: False, Reason: Retention period expired

## 8. Decision Card: When to Use Automated Lifecycle

Based on the module's decision framework.

In [ ]:
print("=" * 70)
print("DECISION CARD: Automated Tenant Lifecycle Management")
print("=" * 70)

print("\n✅ USE AUTOMATED LIFECYCLE WHEN:")
print("  • 10+ tenants with frequent plan changes")
print("  • Regulated industry requiring compliance audit trails")
print("  • Self-service model demanded by market")
print("  • Need to scale lifecycle operations without CSM overhead")

print("\n❌ DON'T USE (Manual/Alternative) WHEN:")
print("  • <10 tenants (manual SQL scripts sufficient)")
print("  • Enterprise-only with <5 annual transitions")
print("  • High-touch relationships requiring personal coordination")
print("  • Instant exports required (<5 min response time)")
print("  • >100 lifecycle events/day (need distributed queue system)")

print("\n🔄 ALTERNATIVES:")
print("  1. Manual CSM-Driven: For enterprise high-touch")
print("     → Lower error risk but higher overhead")
print("  ")
print("  2. Upgrade-Only: Forbid downgrades entirely")
print("     → Simpler but impacts retention")
print("  ")
print("  3. Hard Deletes: No retention period")
print("     → Violates GDPR recovery rights (non-regulated only)")
print("  ")
print("  4. Managed Platforms: BuilderKit/Supabase")
print("     → Less control but turnkey solution")

print("\n💰 COST CONSIDERATIONS (per 100 events/month):")
print("  • Celery workers: ~$50/month")
print("  • Redis: ~$15/month")
print("  • Storage (exports): ~$10/month")
print("  • Total: ~$75/month")

print("\n📊 KEY METRICS TO MONITOR:")
print("  • State transition failures")
print("  • Upgrade/downgrade duration")
print("  • Export completion rate")
print("  • Deletion verification success")
print("  • Reactivation success rate")

print("\n" + "=" * 70)

# Expected:
# Decision card displays with all sections

## Summary

You've completed Module 12.4: Tenant Lifecycle Management. You now understand:\n\n- **State Machine**: 8 states with validated transitions\n- **Upgrades**: Resource provisioning before billing (no service interruption)\n- **Downgrades**: Usage validation to prevent data loss\n- **Data Export**: GDPR-compliant chunked exports with checksums\n- **Soft Deletion**: 30-90 day retention with verification\n- **Reactivation**: Win-back workflows with state conflict resolution\n- **Failure Modes**: 5 critical scenarios and their fixes\n- **Decision Framework**: When to automate vs. manual CSM-driven\n\n### Next Steps\n\n1. **Run the API**: `python app.py` to test endpoints\n2. **Run tests**: `pytest tests_smoke.py -v`\n3. **Explore alternatives**: Review the Decision Card for your use case\n4. **Production deployment**: See README.md for checklist\n\n### Key Takeaways\n\n- Always provision resources **before** billing changes\n- Validate usage before downgrades to prevent data loss\n- Use soft-deletes with retention for GDPR compliance\n- Monitor state transition failures and export completion rates\n- Choose automation based on tenant count and compliance needs\n\n**Module completed!** 🎉